<H1>Neighbourhoods in Toronto</H1>

<h2>Part I</h2>

Lets import required packages

In [46]:
import pandas as pd

Read html page into panda

In [47]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = pd.read_html(url)

In [48]:
df = page[0]
df.shape

(180, 3)

In [49]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove Not assigned Borough rows

In [50]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
df.shape

(103, 3)

Check any duplicate Postal Codes

In [51]:
len(df['Postal Code'].unique()) != len(df['Postal Code'])

False

There are no any duplicate Postal Code rows

In [52]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check any not assigned Neighborhoods in data frame

In [55]:
df["Neighborhood"].isna().sum()

0

There are no Neighborhoods which are not assigned or NaN

In [56]:
#Check Shape of Dataframe
df.shape

(103, 3)

########################################################################################

<h2>Part II</h2>